# Coursera Capstone Project
This project makes use of locating the suitable location to open a restaurant in Mumbai, India.
1. Import libraries
2. Data handling and Cleaning
3. Using Foursquare API
4. Collect information about the venues
5. Clustering of data
6. Observations

1. **IMPORT LIBRARIES**

In [140]:
import pandas as pd
import numpy as np
#!pip install geopy
from geopy.geocoders import Nominatim
#!pip install geocoder
#!pip install folium
import folium
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
#!pip install sklearn
from sklearn.cluster import KMeans
print('Libraries imported!')

Libraries imported!


2. **DATA CLEANING AND HANDLING**

In [30]:
#read data
df = pd.read_csv(r'C:\Users\admin\Desktop\zomato_res_final.csv')

remove the duplicates: Restaurant_Name

In [31]:
df = df.drop_duplicates(subset= ['Restaurant_Name']).reset_index().drop('index',axis = 1)

In [32]:
df.head()

,Additional_outlet_count,Call,Cost_for_two(Rs.),Cuisines,Features,Home_Delivery,Operational_hours,Restaurant_Location,Restaurant_Name,Restaurant_Type,...,Delivery_Time(mins),Rating,Votes,Rating_Category,Operational_after_Midnight,Cuisine_count,Feature_Count,Res_Type_Count,Competitors_in_Location,Score
0,1.0,True,1500,"Finger Food, Continental, European, Italian","Food Hygiene Rated Restaurants In Mumbai, Best...",False,12noon – 1am (Mon-Sun),Kamala Mills Compound,Lord of the Drinks,"Lounge,Casual Dining",...,NaN,4.9,1326.0,Excellent,True,4,2,2,19.0,48.000806
1,1.0,True,800,Pizza,"Value For Money, Best of Mumbai",False,11am – 12:30AM (Mon-Sun),Malad West,Joey's Pizza,Quick Bites,...,NaN,4.6,5974.0,Excellent,True,1,2,1,209.0,71.950295
2,NaN,True,2500,Seafood,"Super Seafood, Best of Mumbai",False,"Closed (Mon),12noon – 3pm, 7pm – 12midnight...",Bandra West,Bastian,"Casual Dining,Bar",...,NaN,4.5,1438.0,Excellent,False,1,2,2,241.0,43.160370
3,NaN,True,1800,"Finger Food, Continental","Where's The Party?, Best of Mumbai, Food Hygie...",False,12noon – 1am (Mon-Sun),Lower Parel,Tamasha,"Lounge,Bar",...,NaN,4.9,3275.0,Excellent,True,2,3,2,125.0,59.778427
4,2.0,True,450,"North Indian, Street Food, Fast Food, Chinese",NaN,True,"12noon – 4pm, 7pm – 11:45pm (Mon-Sun)",Vashi,Bhagat Tarachand,Casual Dining,...,45.0,4.1,1422.0,Very Good,False,4,1,1,116.0,37.546442


EXPLORATORY DATA ANALYSIS

Q - How many different restaurants are there in Mumbai

In [33]:
df[['Restaurant_Name']].count().sort_values()

Restaurant_Name    6181
dtype: int64

Thus, there are 6181 restaurants in Mumbai according to Zomato **NOTE-** Since the amount of restaurants are too high in number, the limit for calls in Foursquare API are easily exhausted while using the 'Sandbox' account. Thus, I have manually removed a lot of cells and only included 100 tuples from the original dataset so that the Foursquare API can be accessed easily.

Now, reading the data again, we have:


In [31]:
#read data
df = pd.read_csv(r'C:\Users\admin\Desktop\zomato_res_final.csv')

Q - Find average rating for each neighbourhood

In [32]:
df.groupby(['Restaurant_Location'])['Rating'].mean().sort_values()

Restaurant_Location
Breach Candy             3.800000
Kandivali West           3.800000
Sakinaka                 3.950000
Worli                    4.000000
Byculla                  4.000000
Charni Road              4.000000
Dadar East               4.000000
Vashi                    4.100000
Marine Lines             4.100000
Mahakali                 4.100000
Jogeshwari               4.100000
Powai                    4.162500
Colaba                   4.180000
Chowpatty                4.200000
Goregaon West            4.200000
Andheri West             4.222222
Churchgate               4.233333
Khar                     4.250000
Sion                     4.300000
Malad West               4.300000
Juhu                     4.320000
Bandra West              4.327273
Fort                     4.350000
Veera Desai Area         4.375000
Marol                    4.400000
Santacruz West           4.400000
Borivali West            4.400000
Flea Bazaar Café         4.400000
Kamala Mills Compound    4.5

Thus, Lower Parel is rated the highest

Q - Find the number of restaurants in each neighbourhood

In [33]:
df.groupby(['Restaurant_Location'])['Restaurant_Name'].count().sort_values()

Restaurant_Location
Jogeshwari                1
Vashi                     1
Sion                      1
Santacruz West            1
Marine Lines              1
Mahakali                  1
Lower Parel               1
Kandivali West            1
Goregaon West             1
Flea Bazaar Café          1
Dadar West                1
Worli                     1
Cuffe Parade              1
Breach Candy              1
Dadar East                1
Charni Road               1
Chowpatty                 1
Byculla                   1
Fort                      2
Malad West                2
Sakinaka                  2
Borivali West             2
Churchgate                3
Marol                     3
Veera Desai Area          4
Khar                      4
Kamala Mills Compound     5
Colaba                    5
Powai                     8
Andheri West              9
Juhu                     10
Bandra West              11
Bandra Kurla Complex     12
Name: Restaurant_Name, dtype: int64

Thus, Bandra Kurla Complex has the most number of restaurants

#### Now, lets get the location of each neighbourhood ie determine latitude and longitude of each neighbourhood

In [34]:
location = [x for x in df['Restaurant_Location'].unique().tolist() if type(x) == str]
latitude = []
longitude =  []
for i in range(0, len(location)):
    if(type(location[i]) == str):
        ctr=0
        while True:
            try:
                address = location[i] + ', Mumbai, India'
                geolocator = Nominatim(user_agent="my-application")
                loc = geolocator.geocode(address)
                latitude.append(loc.latitude)
                longitude.append(loc.longitude)
                print('The geograpical coordinate of location are {}, {}.'.format(loc.latitude, loc.longitude))
            except:
                ctr+=1
                if(ctr==20):
                    print(i)
                    latitude.append(address)
                    longitude.append(address)
                    break
                continue
            break

0
The geograpical coordinate of location are 19.1840129, 72.8412155.
The geograpical coordinate of location are 19.0583358, 72.8302669.
The geograpical coordinate of location are 19.0029881, 72.8303219.
The geograpical coordinate of location are 19.075713, 73.0003541.
5
The geograpical coordinate of location are 19.067115, 72.8657245.
The geograpical coordinate of location are 19.1070215, 72.8275275.
The geograpical coordinate of location are 18.9136406, 72.8209295.
The geograpical coordinate of location are 19.1172495, 72.833968.
The geograpical coordinate of location are 19.1658979, 72.9619708.
The geograpical coordinate of location are 19.1187195, 72.9073476.
The geograpical coordinate of location are 19.0162531, 72.8522274.
The geograpical coordinate of location are 19.084877, 72.8349727.
The geograpical coordinate of location are 18.9332665, 72.8345146.
The geograpical coordinate of location are 19.1338237, 72.8353808.
The geograpical coordinate of location are 19.0724575, 72.8337

In [35]:
df['location_latitude'] = df['Restaurant_Location'].map(dict(zip(location, latitude)))
df['location_longitude'] = df['Restaurant_Location'].map(dict(zip(location, longitude)))

Manually adding latitudes and longitudes of places(Kamala mills compound, Flea Bazaar Cafe, ... etc) for which geolocator does not work.

In [36]:
df['location_latitude'].replace('Kamala Mills Compound, Mumbai, India',19.0039, inplace = True)
df['location_longitude'].replace('Kamala Mills Compound, Mumbai, India',72.8275, inplace = True)
df['location_latitude'].replace('Mohammad Ali Road, Mumbai, India',18.955868 , inplace = True)
df['location_longitude'].replace('Mohammad Ali Road, Mumbai, India',72.83288 , inplace = True)
df['location_latitude'].replace('Flea Bazaar Café, Mumbai, India',19.005876  , inplace = True)
df['location_longitude'].replace('Flea Bazaar Café, Mumbai, India',72.826458 , inplace = True)
df['location_latitude'].replace('Mumbai CST Area, Mumbai, India',18.939821  , inplace = True)
df['location_longitude'].replace('Mumbai CST Area, Mumbai, India',72.835468 , inplace = True)
df['location_latitude'].replace('Near Andheri West Station, Mumbai, India',19.127567   , inplace = True)
df['location_longitude'].replace('Near Andheri West Station, Mumbai, India',72.829761 , inplace = True)
df['location_latitude'].replace('Near Andheri East Station, Mumbai, India',19.127567   , inplace = True)
df['location_longitude'].replace('Near Andheri East Station, Mumbai, India',72.829761 , inplace = True)
df['location_latitude'].replace('Dadar Shivaji Park, Mumbai, India',19.027192   , inplace = True)
df['location_longitude'].replace('Dadar Shivaji Park, Mumbai, India',72.83847 , inplace = True)
df['location_latitude'].replace('Kalwa, Mumbai, India',19.194385  , inplace = True)
df['location_longitude'].replace('Kalwa, Mumbai, India',72.999201 , inplace = True)
df['location_latitude'].replace('Dadar Shivaji Park, Mumbai, India',19.027192   , inplace = True)
df['location_longitude'].replace('Dadar Shivaji Park, Mumbai, India',72.83847 , inplace = True)
df['location_latitude'].replace('Ghodbunder Road, Mumbai, India',19.243896    , inplace = True)
df['location_longitude'].replace('Ghodbunder Road, Mumbai, India',72.97594 , inplace = True)
df['location_latitude'].replace('Ulhasnagar, Mumbai, India',19.2215115 , inplace = True)
df['location_longitude'].replace('Ulhasnagar, Mumbai, India',73.16446280000002, inplace = True)
df['location_latitude'].replace('Dombivali East, Mumbai, India',19.205716     , inplace = True)
df['location_longitude'].replace('Dombivali East, Mumbai, India',73.09553 , inplace = True)
df['location_latitude'].replace('Dombivali West, Mumbai, India',19.215761   , inplace = True)
df['location_longitude'].replace('Dombivali West, Mumbai, India',73.072993 , inplace = True)
df['location_latitude'].replace('Dombivali, Mumbai, India',19.2094006    , inplace = True)
df['location_longitude'].replace('Dombivali, Mumbai, India',73.0939483 , inplace = True)
df['location_latitude'].replace('Ambernath, Mumbai, India',19.182517    , inplace = True)
df['location_longitude'].replace('Ambernath, Mumbai, India',73.192602 , inplace = True)
df['location_latitude'].replace('Panch Pakhadi, Mumbai, India',19.195473     , inplace = True)
df['location_longitude'].replace('Panch Pakhadi, Mumbai, India',72.965249 , inplace = True)
df['location_latitude'].replace('Alibaug, Mumbai, India',18.6553938      , inplace = True)
df['location_longitude'].replace('Alibaug, Mumbai, India',72.86708190000002, inplace = True)
df['location_latitude'].replace('Nalasopara, Mumbai, India',19.432762       , inplace = True)
df['location_longitude'].replace('Nalasopara, Mumbai, India',72.84564, inplace = True)
df=df.dropna(subset=['location_longitude'])
df=df.dropna(subset=['location_latitude'])
df['location_latitude'] = pd.to_numeric(df['location_latitude'])
df['location_longitude'] = pd.to_numeric(df['location_longitude'])

In [37]:
df1 = df[['Restaurant_Location','location_latitude','location_longitude']]
df1.head()

,Restaurant_Location,location_latitude,location_longitude
0,Kamala Mills Compound,19.003900,72.827500
1,Malad West,19.184013,72.841216
2,Bandra West,19.058336,72.830267
3,Lower Parel,19.002988,72.830322
4,Vashi,19.075713,73.000354


Neighbourhood by rating

In [38]:
ratedf = df.groupby(['Restaurant_Location'])['location_latitude', 'location_longitude'].first()
ratedf['Rating'] = df.groupby(['Restaurant_Location'])['Rating'].mean().sort_values()
ratedf

,location_latitude,location_longitude,Rating
Restaurant_Location,,,
Andheri West,19.117249,72.833968,4.222222
Bandra Kurla Complex,19.067115,72.865724,4.500000
Bandra West,19.058336,72.830267,4.327273
Borivali West,19.229456,72.847990,4.400000
Breach Candy,18.971896,72.804032,3.800000
Byculla,18.976622,72.832794,4.000000
Charni Road,18.952758,72.817417,4.000000
Chowpatty,19.149536,72.789056,4.200000
Churchgate,18.935957,72.827340,4.233333


#### Create a map of Mumbai with Restaurant Locations on it

In [39]:
# get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [40]:
# create map of Mumbai using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=12)

# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the restaurants and add each to the incidents feature group
for lat, lng, loc, in zip(df.location_latitude, df.location_longitude,df.Restaurant_Location):
    incidents.add_child(
        folium.map.Marker(
            [lat, lng],
            radius=1, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
map_kl.add_child(incidents)
    
map_kl

3. **USING FOURSQUARE API**

In [41]:
# define Foursquare Credentials and Version
CLIENT_ID = 'JQ5KMONKCXLVQKE2NOIYAZUAOVNGTTVVKKOS4EZHJ10QYKDL' # your Foursquare ID
CLIENT_SECRET = 'F2BRGJEWE4IIWGL1KDFR2R4QLA5RZWEUON4ZQMS0B2HORAWX' # your Foursquare Secret
VERSION = '20191010' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JQ5KMONKCXLVQKE2NOIYAZUAOVNGTTVVKKOS4EZHJ10QYKDL
CLIENT_SECRET:F2BRGJEWE4IIWGL1KDFR2R4QLA5RZWEUON4ZQMS0B2HORAWX


Lets get top 100 venues within 2000 meter radius

In [45]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['location_latitude'], df['location_longitude'], df['Restaurant_Location']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [110]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Venue_Location', 'Latitude', 'Longitude', 'Venue_Name', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']

print(venues_df.shape)
venues_df.head()

(7143, 7)


,Venue_Location,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Kamala Mills Compound,19.0039,72.8275,Bombay Canteen,19.003410,72.827816,Indian Restaurant
1,Kamala Mills Compound,19.0039,72.8275,Jai Hind Lunch Home,19.002183,72.829512,Seafood Restaurant
2,Kamala Mills Compound,19.0039,72.8275,SMAAASH,19.004472,72.826114,Athletics & Sports
3,Kamala Mills Compound,19.0039,72.8275,Koko's Kitchen,19.002569,72.827672,Asian Restaurant
4,Kamala Mills Compound,19.0039,72.8275,Café Zoe,19.000226,72.830252,Café


Q- Lets check how many venues were returned based on the location

In [111]:
venues_df.groupby(["Venue_Location"]).count()


,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
Venue_Location,,,,,,
Andheri West,700,700,700,700,700,700
Bandra Kurla Complex,693,693,693,693,693,693
Bandra West,800,800,800,800,800,800
Borivali West,200,200,200,200,200,200
Byculla,70,70,70,70,70,70
Chowpatty,9,9,9,9,9,9
Churchgate,300,300,300,300,300,300
Colaba,400,400,400,400,400,400
Cuffe Parade,100,100,100,100,100,100


Q - Find all unique categories from returned venues

In [112]:
print('There are {} uniques categories.'.format(len(venues_df['Venue_Category'].unique())))

There are 185 uniques categories.


In [113]:
venues_df['Venue_Category'].unique()


array(['Indian Restaurant', 'Seafood Restaurant', 'Athletics & Sports',
       'Asian Restaurant', 'Café', 'Lounge', 'Italian Restaurant',
       'Japanese Restaurant', 'Flea Market',
       'Molecular Gastronomy Restaurant', 'Pub', 'Shopping Mall',
       'Thai Restaurant', 'Burger Joint', 'Dessert Shop', 'Hotel',
       'Office', 'Restaurant', 'Juice Bar', 'Deli / Bodega',
       'Pizza Place', 'Cupcake Shop', 'Comedy Club', 'Hotel Bar',
       'Gym / Fitness Center', 'Scenic Lookout', 'Cosmetics Shop',
       'Toy / Game Store', 'Multiplex', 'Movie Theater', 'Gourmet Shop',
       'Spa', 'Coffee Shop', 'Bar', 'Bakery', 'Donut Shop',
       'Clothing Store', 'Sandwich Place', 'Music Venue',
       'Ice Cream Shop', 'Recreation Center', 'Chinese Restaurant',
       'Mediterranean Restaurant', 'Snack Place', 'French Restaurant',
       'Plaza', 'Brewery', 'Roof Deck', 'Fast Food Restaurant',
       'Cocktail Bar', 'Theater', 'Tea Room',
       'Vegetarian / Vegan Restaurant', 'Garden',

In [115]:
#Check if there are restaurants in those venues
"Restaurant" in venues_df['Venue_Category'].unique()

True

4. **COLLECT INFORMATION ABOUT THE VENUES**

In [116]:
# one hot encoding
onehot = pd.get_dummies(venues_df[['Venue_Category']], prefix="", prefix_sep="")

# add Venue_Location column back to dataframe
onehot['Venue_Location'] = venues_df['Venue_Location'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)
onehot.head()

(7143, 186)


,Venue_Location,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,...,Toy / Game Store,Track,Track Stadium,Train,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Kamala Mills Compound,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Kamala Mills Compound,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kamala Mills Compound,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Kamala Mills Compound,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kamala Mills Compound,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
grouped = onehot.groupby(["Venue_Location"]).mean().reset_index()

print(grouped.shape)
grouped

(28, 186)


,Venue_Location,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,...,Toy / Game Store,Track,Track Stadium,Train,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Andheri West,0.010000,0.000000,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.010000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.030000,0.00,0.010000,0.000000,0.000000
1,Bandra Kurla Complex,0.000000,0.000000,0.000000,0.00,0.012987,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.012987,0.000000,0.00,0.000000,0.000000,0.000000
2,Bandra West,0.000000,0.000000,0.020000,0.00,0.000000,0.030000,0.000000,0.000000,0.010000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Borivali West,0.000000,0.000000,0.010000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.020000,0.020000,0.00,0.000000,0.000000,0.000000
4,Byculla,0.000000,0.014286,0.000000,0.00,0.014286,0.014286,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.014286,0.000000,0.000000,0.00,0.000000,0.000000,0.014286
5,Chowpatty,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Churchgate,0.000000,0.000000,0.000000,0.01,0.000000,0.020000,0.010000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.000000
7,Colaba,0.000000,0.000000,0.000000,0.01,0.000000,0.030000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Cuffe Parade,0.000000,0.000000,0.000000,0.01,0.000000,0.020000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.000000,0.000000,0.000000
9,Dadar East,0.000000,0.000000,0.010989,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.010989,0.000000,0.000000,0.021978,0.032967,0.00,0.000000,0.000000,0.000000


In [93]:
#determine the number of restaurants 
len(grouped[grouped["Restaurant"] > 0])

23

Create new dataframe for restaurants/cafes,etc. only

In [123]:
rest = grouped[['Venue_Location','Restaurant',"American Restaurant",'Indian Restaurant', 'Seafood Restaurant',
                'Asian Restaurant', 'Café', 'Italian Restaurant','Japanese Restaurant', 'Molecular Gastronomy Restaurant',
       'Thai Restaurant', 'Burger Joint','Restaurant','Pizza Place','Sandwich Place', 'Chinese Restaurant','Mediterranean Restaurant', 'Snack Place', 'French Restaurant', 'Fast Food Restaurant',
        'Cocktail Bar','Vegetarian / Vegan Restaurant','Burrito Place','BBQ Joint', 'American Restaurant','Dim Sum Restaurant', 'Breakfast Spot','Salad Place','German Restaurant',
       'Sushi Restaurant','Modern European Restaurant','Gluten-free Restaurant', 'Korean Restaurant','Fried Chicken Joint','Middle Eastern Restaurant',
       'Indian Chinese Restaurant', 'Irani Cafe','Goan Restaurant', 'Noodle House', 'Diner','Brazilian Restaurant', 'Food Court','Spanish Restaurant',
       'Australian Restaurant', 'Chaat Place', 'Falafel Restaurant', 'New American Restaurant','Mughlai Restaurant','Moroccan Restaurant', 'North Indian Restaurant',
       'South Indian Restaurant','Multicuisine Indian Restaurant', 'Maharashtrian Restaurant','Bistro','Parsi Restaurant', 'Comfort Food Restaurant',
       'Sports Bar', 'Bengali Restaurant', 'Mexican Restaurant','Greek Restaurant', 'South American Restaurant',
        'Steakhouse', 'Dumpling Restaurant']]

In [124]:
rest.head()

,Venue_Location,Restaurant,American Restaurant,Indian Restaurant,Seafood Restaurant,Asian Restaurant,Café,Italian Restaurant,Japanese Restaurant,Molecular Gastronomy Restaurant,...,Bistro,Parsi Restaurant,Comfort Food Restaurant,Sports Bar,Bengali Restaurant,Mexican Restaurant,Greek Restaurant,South American Restaurant,Steakhouse,Dumpling Restaurant
0,Andheri West,0.010000,0.01,0.140000,0.040000,0.030000,0.050000,0.010000,0.000000,0.00,...,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0
1,Bandra Kurla Complex,0.000000,0.00,0.142857,0.012987,0.000000,0.064935,0.051948,0.012987,0.00,...,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0
2,Bandra West,0.020000,0.00,0.090000,0.040000,0.030000,0.080000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0
3,Borivali West,0.070000,0.00,0.110000,0.000000,0.010000,0.090000,0.010000,0.000000,0.01,...,0.0,0.0,0.0,0.01,0.000000,0.01,0.0,0.0,0.0,0.0
4,Byculla,0.042857,0.00,0.142857,0.000000,0.014286,0.028571,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.00,0.014286,0.00,0.0,0.0,0.0,0.0


5. **CLUSTERING OF DATA**

In [131]:
# set number of clusters
kclusters = 3

clustering = rest.drop(["Venue_Location"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 0, 1, 2, 2, 2, 2, 1])

In [132]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
merged = rest.copy()

# add clustering labels
merged["Cluster Labels"] = kmeans.labels_

In [133]:
merged.head()

,Venue_Location,Restaurant,American Restaurant,Indian Restaurant,Seafood Restaurant,Asian Restaurant,Café,Italian Restaurant,Japanese Restaurant,Molecular Gastronomy Restaurant,...,Parsi Restaurant,Comfort Food Restaurant,Sports Bar,Bengali Restaurant,Mexican Restaurant,Greek Restaurant,South American Restaurant,Steakhouse,Dumpling Restaurant,Cluster Labels
0,Andheri West,0.010000,0.01,0.140000,0.040000,0.030000,0.050000,0.010000,0.000000,0.00,...,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,1
1,Bandra Kurla Complex,0.000000,0.00,0.142857,0.012987,0.000000,0.064935,0.051948,0.012987,0.00,...,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,1
2,Bandra West,0.020000,0.00,0.090000,0.040000,0.030000,0.080000,0.000000,0.000000,0.00,...,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,2
3,Borivali West,0.070000,0.00,0.110000,0.000000,0.010000,0.090000,0.010000,0.000000,0.01,...,0.0,0.0,0.01,0.000000,0.01,0.0,0.0,0.0,0.0,0
4,Byculla,0.042857,0.00,0.142857,0.000000,0.014286,0.028571,0.000000,0.000000,0.00,...,0.0,0.0,0.00,0.014286,0.00,0.0,0.0,0.0,0.0,1


In [134]:
# merge to add latitude/longitude for each neighborhood
merged = merged.join(df.set_index("Restaurant_Location"), on="Venue_Location")

print(merged.shape)
merged.head() # check the last columns!

(94, 87)


,Venue_Location,Restaurant,American Restaurant,Indian Restaurant,Seafood Restaurant,Asian Restaurant,Café,Italian Restaurant,Japanese Restaurant,Molecular Gastronomy Restaurant,...,Votes,Rating_Category,Operational_after_Midnight,Cuisine_count,Feature_Count,Res_Type_Count,Competitors_in_Location,Score,location_latitude,location_longitude
0,Andheri West,0.01,0.01,0.14,0.04,0.03,0.05,0.01,0.0,0.0,...,2107,Good,True,3,2,1,295,38.927211,19.117249,72.833968
0,Andheri West,0.01,0.01,0.14,0.04,0.03,0.05,0.01,0.0,0.0,...,1796,Very Good,False,3,1,2,295,39.806488,19.117249,72.833968
0,Andheri West,0.01,0.01,0.14,0.04,0.03,0.05,0.01,0.0,0.0,...,2820,Good,True,5,1,1,295,43.235802,19.117249,72.833968
0,Andheri West,0.01,0.01,0.14,0.04,0.03,0.05,0.01,0.0,0.0,...,2054,Very Good,True,2,1,1,295,45.503489,19.117249,72.833968
0,Andheri West,0.01,0.01,0.14,0.04,0.03,0.05,0.01,0.0,0.0,...,1676,Very Good,False,2,1,1,295,43.219271,19.117249,72.833968


In [135]:
# sort the results by Cluster Labels
print(merged.shape)
merged.sort_values(["Cluster Labels"], inplace=True)
merged

(94, 87)


,Venue_Location,Restaurant,American Restaurant,Indian Restaurant,Seafood Restaurant,Asian Restaurant,Café,Italian Restaurant,Japanese Restaurant,Molecular Gastronomy Restaurant,...,Votes,Rating_Category,Operational_after_Midnight,Cuisine_count,Feature_Count,Res_Type_Count,Competitors_in_Location,Score,location_latitude,location_longitude
3,Borivali West,0.070000,0.00,0.110000,0.000000,0.010000,0.090000,0.010000,0.000000,0.010000,...,1062,Very Good,True,4,1,1,193,35.370996,19.229456,72.847990
3,Borivali West,0.070000,0.00,0.110000,0.000000,0.010000,0.090000,0.010000,0.000000,0.010000,...,660,Excellent,False,3,3,1,193,41.217611,19.229456,72.847990
15,Kandivali West,0.036145,0.00,0.132530,0.012048,0.000000,0.060241,0.024096,0.000000,0.000000,...,1288,Good,False,5,1,1,131,32.598762,19.208380,72.842227
18,Mahakali,0.028571,0.00,0.171429,0.000000,0.028571,0.057143,0.000000,0.000000,0.000000,...,1053,Very Good,True,6,1,1,84,35.316610,19.165898,72.961971
12,Goregaon West,0.030000,0.00,0.080000,0.030000,0.000000,0.070000,0.030000,0.000000,0.000000,...,1679,Very Good,True,2,1,2,86,40.478779,19.163328,72.841200
0,Andheri West,0.010000,0.01,0.140000,0.040000,0.030000,0.050000,0.010000,0.000000,0.000000,...,2107,Good,True,3,2,1,295,38.927211,19.117249,72.833968
14,Kamala Mills Compound,0.040000,0.00,0.120000,0.030000,0.030000,0.040000,0.020000,0.010000,0.020000,...,1054,Excellent,True,1,2,2,19,46.357136,19.003900,72.827500
14,Kamala Mills Compound,0.040000,0.00,0.120000,0.030000,0.030000,0.040000,0.020000,0.010000,0.020000,...,1326,Excellent,True,4,2,2,19,48.000806,19.003900,72.827500
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.000000,...,2358,Very Good,True,1,3,1,100,44.581911,19.107021,72.827528
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.000000,...,911,Very Good,True,3,2,1,100,37.217138,19.107021,72.827528


In [136]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['location_latitude'], merged['location_longitude'], merged['Venue_Location'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [137]:
merged.loc[merged['Cluster Labels'] == 0]


,Venue_Location,Restaurant,American Restaurant,Indian Restaurant,Seafood Restaurant,Asian Restaurant,Café,Italian Restaurant,Japanese Restaurant,Molecular Gastronomy Restaurant,...,Votes,Rating_Category,Operational_after_Midnight,Cuisine_count,Feature_Count,Res_Type_Count,Competitors_in_Location,Score,location_latitude,location_longitude
3,Borivali West,0.070000,0.0,0.110000,0.000000,0.010000,0.090000,0.010000,0.0,0.01,...,1062,Very Good,True,4,1,1,193,35.370996,19.229456,72.847990
3,Borivali West,0.070000,0.0,0.110000,0.000000,0.010000,0.090000,0.010000,0.0,0.01,...,660,Excellent,False,3,3,1,193,41.217611,19.229456,72.847990
15,Kandivali West,0.036145,0.0,0.132530,0.012048,0.000000,0.060241,0.024096,0.0,0.00,...,1288,Good,False,5,1,1,131,32.598762,19.208380,72.842227
18,Mahakali,0.028571,0.0,0.171429,0.000000,0.028571,0.057143,0.000000,0.0,0.00,...,1053,Very Good,True,6,1,1,84,35.316610,19.165898,72.961971
12,Goregaon West,0.030000,0.0,0.080000,0.030000,0.000000,0.070000,0.030000,0.0,0.00,...,1679,Very Good,True,2,1,2,86,40.478779,19.163328,72.841200


In [138]:
merged.loc[merged['Cluster Labels'] == 1]


,Venue_Location,Restaurant,American Restaurant,Indian Restaurant,Seafood Restaurant,Asian Restaurant,Café,Italian Restaurant,Japanese Restaurant,Molecular Gastronomy Restaurant,...,Votes,Rating_Category,Operational_after_Midnight,Cuisine_count,Feature_Count,Res_Type_Count,Competitors_in_Location,Score,location_latitude,location_longitude
0,Andheri West,0.010000,0.01,0.140000,0.040000,0.030000,0.050000,0.010000,0.000000,0.00,...,2107,Good,True,3,2,1,295,38.927211,19.117249,72.833968
14,Kamala Mills Compound,0.040000,0.00,0.120000,0.030000,0.030000,0.040000,0.020000,0.010000,0.02,...,1054,Excellent,True,1,2,2,19,46.357136,19.003900,72.827500
14,Kamala Mills Compound,0.040000,0.00,0.120000,0.030000,0.030000,0.040000,0.020000,0.010000,0.02,...,1326,Excellent,True,4,2,2,19,48.000806,19.003900,72.827500
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.00,...,2358,Very Good,True,1,3,1,100,44.581911,19.107021,72.827528
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.00,...,911,Very Good,True,3,2,1,100,37.217138,19.107021,72.827528
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.00,...,2900,Excellent,True,3,1,1,100,51.995096,19.107021,72.827528
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.00,...,1347,Excellent,True,1,1,2,100,42.610465,19.107021,72.827528
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.00,...,379,Very Good,True,2,1,2,100,32.623003,19.107021,72.827528
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.00,...,1308,Good,True,4,1,1,100,32.719620,19.107021,72.827528
13,Juhu,0.010000,0.01,0.140000,0.040000,0.020000,0.040000,0.010000,0.000000,0.00,...,2629,Very Good,False,3,1,1,100,46.219538,19.107021,72.827528


In [139]:
merged.loc[merged['Cluster Labels'] == 2]


,Venue_Location,Restaurant,American Restaurant,Indian Restaurant,Seafood Restaurant,Asian Restaurant,Café,Italian Restaurant,Japanese Restaurant,Molecular Gastronomy Restaurant,...,Votes,Rating_Category,Operational_after_Midnight,Cuisine_count,Feature_Count,Res_Type_Count,Competitors_in_Location,Score,location_latitude,location_longitude
16,Khar,0.000000,0.000000,0.070000,0.040000,0.020000,0.080000,0.010000,0.010000,0.000000,...,1729,Very Good,True,2,1,2,98,40.780924,19.072457,72.833707
19,Malad West,0.012987,0.025974,0.090909,0.000000,0.000000,0.038961,0.025974,0.000000,0.000000,...,1521,Very Good,False,6,1,2,209,36.765379,19.184013,72.841216
19,Malad West,0.012987,0.025974,0.090909,0.000000,0.000000,0.038961,0.025974,0.000000,0.000000,...,5974,Excellent,True,1,2,1,209,71.950295,19.184013,72.841216
23,Santacruz West,0.010204,0.000000,0.102041,0.010204,0.010204,0.040816,0.051020,0.010204,0.000000,...,2838,Very Good,True,4,2,2,37,50.241126,19.084877,72.834973
25,Vashi,0.035294,0.011765,0.082353,0.035294,0.011765,0.035294,0.023529,0.000000,0.000000,...,1422,Very Good,False,4,1,1,116,37.546442,19.075713,73.000354
26,Veera Desai Area,0.000000,0.010000,0.110000,0.000000,0.010000,0.070000,0.030000,0.000000,0.000000,...,1818,Very Good,True,2,1,2,36,42.698053,19.133824,72.835381
26,Veera Desai Area,0.000000,0.010000,0.110000,0.000000,0.010000,0.070000,0.030000,0.000000,0.000000,...,2414,Excellent,False,3,4,2,36,53.196176,19.133824,72.835381
26,Veera Desai Area,0.000000,0.010000,0.110000,0.000000,0.010000,0.070000,0.030000,0.000000,0.000000,...,3363,Very Good,False,6,2,1,36,52.034340,19.133824,72.835381
16,Khar,0.000000,0.000000,0.070000,0.040000,0.020000,0.080000,0.010000,0.010000,0.000000,...,3822,Very Good,True,2,2,2,98,53.428723,19.072457,72.833707
6,Churchgate,0.020000,0.000000,0.100000,0.020000,0.020000,0.080000,0.020000,0.010000,0.000000,...,5446,Very Good,True,3,3,1,24,64.621711,18.935957,72.827340


6. **OBSERVATIONS**

As we can see from the above clusters, cluster 0 has the least number of restaurants in Mumbai. Cluster 1 and cluster 2 have a large number of restaurants in the city, thus, a lot of competition will be faced if the restaurant is opened in either location of cluster 1 or cluster 2. Thus, opening a restaurant in Borivali West, Kandivali West, Goregaon West, or Mahakali will be a safe bet as competition is very low in these areas. Moreover, these areas are not in the core locations of mumbai like BKC, Kamala Mills Compound, Juhu, Powai, etc. Thus, opening a restaurant will be easier and cheaper for any individual. We can also observe that most of the restaurants are of Cafe or Indian restaurant type. Thus, opening a restaurant like American Restaurant or Japanese Restaurant will be more beneficial as it will provide diversity to the area. Thus, opening a restaurant in some other type than Cafe or Indian Restaurant will be more beneficial for any individual